In [25]:
import os
import cv2
import numpy as np
import pandas as pd

import image_processor
import model_builder

from tensorflow.python.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

In [26]:
image_size = (32, 32)
input_size = (*image_size, 1)

In [27]:
df_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
            'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
            'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
            'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
            'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

In [28]:
df_train = image_processor.process_image_data("data/Train", image_size, targets=df_names)

In [29]:
X = df_train.drop(columns=['target'])
y = df_train['target']

In [30]:
X_data = X.values.reshape(-1, *image_size, 1).astype('float32')
y_data = y.factorize()[0]

In [31]:
X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.2, stratify=y_data, random_state=42)

In [32]:
model = model_builder.build_conv_pool_model(input_size, len(df_names))

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(X_train, y_train,
                    epochs=50,
                    batch_size=64,
                    callbacks=[early_stopping],
                    validation_data=(X_val, y_val),
                    verbose=1)

Epoch 1/50
665/665 [==============================] - 16s 24ms/step - loss: 1.1316 - accuracy: 0.7158 - val_loss: 0.5404 - val_accuracy: 0.8288
Epoch 2/50
665/665 [==============================] - 16s 24ms/step - loss: 0.5814 - accuracy: 0.8211 - val_loss: 0.4456 - val_accuracy: 0.8567
Epoch 3/50
665/665 [==============================] - 16s 24ms/step - loss: 0.4508 - accuracy: 0.8498 - val_loss: 0.4222 - val_accuracy: 0.8565
Epoch 4/50
665/665 [==============================] - 15s 22ms/step - loss: 0.3907 - accuracy: 0.8665 - val_loss: 0.3332 - val_accuracy: 0.8828
Epoch 5/50
665/665 [==============================] - 15s 22ms/step - loss: 0.3548 - accuracy: 0.8751 - val_loss: 0.3743 - val_accuracy: 0.8760
Epoch 6/50
665/665 [==============================] - 15s 22ms/step - loss: 0.3308 - accuracy: 0.8819 - val_loss: 0.3350 - val_accuracy: 0.8848
Epoch 7/50
665/665 [==============================] - 15s 22ms/step - loss: 0.3080 - accuracy: 0.8898 - val_loss: 0.3200 - val_accuracy:

In [33]:
for epoch, (val_loss, val_acc) in enumerate(zip(history.history['val_loss'], history.history['val_accuracy']), start=1):
    print(f"Epoch {epoch}: val_loss={val_loss:.4f}, val_accuracy={val_acc:.4f}")

score = model.evaluate(X_val, y_val, verbose=0)
print(f"Final Validation Accuracy: {score[1] * 100:.2f}%")

Epoch 1: val_loss=0.5404, val_accuracy=0.8288
Epoch 2: val_loss=0.4456, val_accuracy=0.8567
Epoch 3: val_loss=0.4222, val_accuracy=0.8565
Epoch 4: val_loss=0.3332, val_accuracy=0.8828
Epoch 5: val_loss=0.3743, val_accuracy=0.8760
Epoch 6: val_loss=0.3350, val_accuracy=0.8848
Epoch 7: val_loss=0.3200, val_accuracy=0.8869
Epoch 8: val_loss=0.3416, val_accuracy=0.8842
Epoch 9: val_loss=0.3177, val_accuracy=0.8874
Epoch 10: val_loss=0.2712, val_accuracy=0.9033
Epoch 11: val_loss=0.3096, val_accuracy=0.8974
Epoch 12: val_loss=0.2778, val_accuracy=0.9029
Epoch 13: val_loss=0.2637, val_accuracy=0.9093
Epoch 14: val_loss=0.2776, val_accuracy=0.9007
Epoch 15: val_loss=0.2818, val_accuracy=0.9000
Epoch 16: val_loss=0.2737, val_accuracy=0.9031
Epoch 17: val_loss=0.2814, val_accuracy=0.9052
Epoch 18: val_loss=0.2580, val_accuracy=0.9111
Epoch 19: val_loss=0.2716, val_accuracy=0.9067
Epoch 20: val_loss=0.2872, val_accuracy=0.9045
Epoch 21: val_loss=0.2724, val_accuracy=0.9083
Epoch 22: val_loss=0.2

In [34]:
test_df = df_test = image_processor.process_image_data("data/TestData", image_size)

In [35]:
X_test = test_df.iloc[:, 1:]
X_test_prepared = X_test.values.reshape(-1, *image_size, 1).astype('float32')

pred = model.predict(X_test_prepared)
predicted_classes = pred.argmax(axis=1)
predicted_labels = [df_names[i] for i in predicted_classes]

test_df['pred'] = predicted_labels

222/222 [==============================] - 1s 4ms/step


In [38]:
model.save("models/model_9133")

INFO:tensorflow:Assets written to: models/model_9133/assets


INFO:tensorflow:Assets written to: models/model_9133/assets


In [37]:
filtered_df = test_df[test_df['pred'] == 'r']
filtered_df[['label', 'pred']]

,label,pred
34,Test3016.png,r
45,Test0286.png,r
50,Test1826.png,r
179,Test0331.png,r
200,Test3162.png,r
...,...,...
6027,Test0306.png,r
6398,Test1020.png,r
6415,Test0464.png,r
6514,Test1592.png,r
